# Predicting EUR

In [ ]:
from itertools import product
from functools import partial
import datetime as dt
import pandas as pd
import satori

In [ ]:
try:
    df = pd.read_csv("../data/history/EUR.csv", index_col=0)
except:
    df = satori.apis.fx.compileHistory(
        name='EUR', 
        days=8000, 
        cooldown=0, 
        saveDuring=True)
df.head(2)

In [ ]:
df.columns

In [ ]:
targets = ['USD', 'CHF', 'GBP', 'CAD', 'AUD', 'NZD', 'JPY',]
times = ['year', 'month', 'day', 'weekday',]

In [ ]:
# drop rows if targets are nan
df = df.drop(index=df[df[targets].isna().any(axis=1)].index)

In [ ]:
# drop all but targets and times
df = df[[*targets, *times]]
df.tail(2)

In [ ]:
# todo: get all the missing days one at a time

In [ ]:
# todo: get high low close in usd too.

In [ ]:
def getData(date=None) -> pd.DataFrame:
    ''' returns most recent price (as of 3pm daily) '''
    rates = satori.apis.fx.getCurrent(date=date)
    targets = ['USD', 'CHF', 'GBP', 'CAD', 'AUD', 'NZD', 'JPY',]
    times = ['year', 'month', 'day', 'weekday',]
    rates = rates.drop(index=df[df[targets].isna().any(axis=1)].index)
    rates = rates[[*targets, *times]]
    return rates

In [ ]:
def validateData(data:pd.DataFrame, existing:pd.DataFrame) -> bool:
    def lastRow():
        return existing.iloc[-1:,:]

    if data.empty or lastRow().equals(data):
        return False
    return True

In [ ]:
# update dataset...
last = df.iloc[-1:,:].index[0]
date = dt.datetime.strptime(last, '%Y-%m-%d').date()
x = 0
while date < dt.datetime.now().date() + dt.timedelta(days=-1):
    x += 1
    date = date + dt.timedelta(days=1)
    row = getData(date=date)
    df = satori.DataManager.defaultAppend(data=row, existing=df, resetIndex=False)
if x > 0:
    df.to_csv('../data/history/EUR.csv')
df

In [ ]:
data = satori.DataManager(
    data=df,
    getData=getData,
    validateData=validateData,
    appendData=partial(satori.DataManager.defaultAppend, resetIndex=False))

In [ ]:
api = satori.apis.google.Google()

In [ ]:
kwargs = {
    'hyperParameters': [
        satori.HyperParameter(
            name='n_estimators',
            value=300,
            kind=int,
            limit=100,
            minimum=200,
            maximum=5000),
        satori.HyperParameter(
            name='learning_rate',
            value=0.3,
            kind=float,
            limit=.05,
            minimum=.01,
            maximum=.1),
        satori.HyperParameter(
            name='max_depth',
            value=6,
            kind=int,
            limit=1,
            minimum=2,
            maximum=10),
        #satori.HyperParameter(
        #    name='gamma',
        #    value=0,
        #    kind=float,
        #    limit=.05,
        #    minimum=0,
        #    maximum=1),
        #satori.HyperParameter(
        #    name='min_child_weight',
        #    value=1,
        #    kind=float,
        #    limit=.1,
        #    minimum=0,
        #    maximum=5),
        #satori.HyperParameter(
        #    name='max_delta_step',
        #    value=0,
        #    kind=float,
        #    limit=.01,
        #    minimum=0,
        #    maximum=1),
        #satori.HyperParameter(
        #    name='scale_pos_weight',
        #    value=1,
        #    kind=float,
        #    limit=.1,
        #    minimum=0,
        #    maximum=5),
        #satori.HyperParameter(
        #    name='base_score',
        #    value=.5,
        #    kind=float,
        #    limit=.05,
        #    minimum=0,
        #    maximum=1),
    ],
    'metrics':  {
        # raw data features
        'raw': satori.ModelManager.rawDataMetric,
        # daily percentage change, 1 day ago, 2 days ago, 3 days ago... 
        **{f'Daily{i}': partial(satori.ModelManager.dailyPercentChangeMetric, yesterday=i) for i in list(range(1, 31))},
        # rolling period transformation percentage change, max of the last 7 days, etc... 
        **{f'Rolling{i}{tx[0:3]}': partial(satori.ModelManager.rollingPercentChangeMetric, window=i, transformation=tx)
            for tx, i in product('sum() max() min() mean() median() std()'.split(), list(range(2, 21)))},
        # rolling period transformation percentage change, max of the last 50 or 70 days, etc... 
        **{f'Rolling{i}{tx[0:3]}': partial(satori.ModelManager.rollingPercentChangeMetric, window=i, transformation=tx)
            for tx, i in product('sum() max() min() mean() median() std()'.split(), list(range(22, 90, 7)))}},
    'override': False}

In [ ]:
learner = satori.Learner(
    cooldown=3,
    recess=3,
    api=api,
    data=data,
    models={
        satori.ModelManager(
            modelPath=f'../models/EUR{currency}.jobLib',
            chosenFeatures=[f'Raw{currency}'],
            pinnedFeatures=[f'Raw{currency}'],
            targetKey=currency,
            **kwargs) 
        for currency in targets
    })

In [ ]:
learner.run(cooldown=0, recess=60*60*23, points=10, view=False)